# Seam carving
- 내용인지 기반 영상 크기 조정
- 영상에서 덜 중요한 경로(seam)을 찾아서 이를 제거(carving)
- insteresting region과 uninteresting region을 나누고 비관심영역을 제거하는 방식

### Interesting region을 어떻게 정의할까
- 화소의 미분값으로 level of activity를 판단
- 만약 어떤 활동이 있다면 미분값이 높아진다.(ex. sobel)


In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [5]:
# draw vertical seam on top of image
def overlay_vertical_seam(img,seam):
    img_seam_overlay = np.copy(img)
    
    x_coords,y_coords = np.transpose([(i,int(j)) for i,j in enumerate(seam)]) # point 추출
    
    img_seam_overlay[x_coords,y_coords] = (0,255,0)
    return img_seam_overlay

In [3]:
def compute_energy_matrix(img):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    sobel_x = cv2.Sobel(gray,cv2.CV_64F,1,0,ksize = 3) # x미분값 계산
    sobel_y = cv2.Sobel(gray,cv2.CV_64F,0,1,ksize = 3)
    
    abs_sobel_x = cv2.convertScaleAbs(sobel_x)
    abs_sobel_y = cv2.convertScaleAbs(sobel_y)
    
    return cv2.addWeighted(abs_sobel_x,0.5,abs_sobel_y,0.5,0)

In [7]:
def find_vertical_seam(img,energy):
    rows,cols = img.shape[:2]
    seam = np.zeros(img.shape[0])
    dist_to = np.zeros(img.shape[:2])+float('inf')
    dist_to[0,:] = np.zeros(img.shape[1])
    edge_to = np.zeros(img.shape[:2])
       
    for row in range(rows-1): 
        for col in range(cols): 
            if col != 0 and dist_to[row+1, col-1] > dist_to[row, col] + energy[row+1, col-1]: 
                dist_to[row+1, col-1] = dist_to[row, col] + energy[row+1, col-1] 
                edge_to[row+1, col-1] = 1 
 
            if dist_to[row+1, col] > dist_to[row, col] + energy[row+1, col]: 
                dist_to[row+1, col] = dist_to[row, col] + energy[row+1, col] 
                edge_to[row+1, col] = 0 
 
            if col != cols-1: 
                if dist_to[row+1, col+1] > dist_to[row, col] + energy[row+1, col+1]: 
                    dist_to[row+1, col+1] = dist_to[row, col] + energy[row+1, col+1] 
                    edge_to[row+1, col+1] = -1
    seam[rows-1] = np.argmin(dist_to[rows-1, :]) 
    for i in (x for x in reversed(range(rows)) if x > 0): 
        seam[i-1] = seam[i] + edge_to[i, int(seam[i])] 
 
    return seam 